In [1]:
import time
import sys
from collections import Counter
import pandas as pd
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
%pylab inline
import pylab
import gzip
from Bio import SeqIO
import seaborn as sns

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
#The inputs here are text files
#If your sequencing files are in fastq, change their file type to .txt and this should work!
f1 = open('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_Mating_Characterization/PreCharacterized_Libraries/binders-100uM_bad_R1_001.txt', "r")
f2 = open('/Users/dyounger/GoogleDrive/UW_Work/PAPER_YSA/Data/NGS/NGS_Mating_Characterization/PreCharacterized_Libraries/binders-100uM_bad_R2_001.txt', "r")

#privide a name for the output csv file:
name='binders_100uM_Bad'

In [3]:
#Read the txt files and grab just the sequences
z1 = f1.readline()
a1=[]
x=1
while True:
    a1.append(f1.readline())
    z1 = f1.readline()
    z1 = f1.readline()
    z1 = f1.readline()
    if z1.strip() == "":
        break
    x=x+1
f1.close()

df=pd.DataFrame(data=a1,columns=['read1'])
len(df.index)

z2 = f2.readline()
a2=[]
x=1
while True:
    a2.append(f2.readline())

    z2 = f2.readline()
    z2 = f2.readline()
    z2 = f2.readline()
    if z2.strip() == "":
        break
    x=x+1
f2.close()

read2df=pd.DataFrame(data=a2,columns=['read2'])
len(read2df.index)
df=df.join(read2df)

In [4]:
# Filter for read quality by look at conserve regions
df=df[df.read1.str.slice(22,37) == 'GATTCGAAAAGCGGC']
df=df[df.read1.str.slice(47,62) == 'GCTTCAATAAAGGAG']
df=df[df.read2.str.slice(22,37) == 'CACAACCAGGTATCG']
df=df[df.read2.str.slice(47,62) == 'ATTAGTTATGTCACG']

# Grab just the barcodes and join them into one dataframe
bc=pd.DataFrame(data=df.read1.str.slice(37,47),columns=['read1'])
read2bc=pd.DataFrame(data=df.read2.str.slice(37,47),columns=['read2'])
bc=bc.join(read2bc)
print len(bc)

192182


In [5]:
#replace the barcodes with the protein they represent
#MATa barcodes
bc = bc.replace(['TCTGAATCAA','TTATTACCAT','GGTTCTATAA','AGTAGATCGT','CTCACGTGTG','AATCCAACGA'],['Bcl2','BclB','BclXL','Bfl1','BclW','Mcl1'])
#MATalpha de novo binder barcodes
bc = bc.replace(['GTCAACTATT','ATACCTGTAC','TGTAACTTGT','GACTACGGGG','GCTATTCTGT','CCGTAAGGCT','GGGTGAGGTG','TGTGGTAATG','GGCGGGTGCG'],['F21','F30D','B+','B-CDP01','B40A','2+','4LVT','X-CDP07','MINDI'])
#MATalpha BH3 peptide barcodes
bc = bc.replace(['GAGAGTACGG','TCGTAAAGCG','AGGTGATCAT','CAGTTTTGTG','AGCTTGACAA','GTAATGTACT','TATCGAGTAT'],['Bim','Noxa','Puma','Bad','Bik','Hrk','Bmf'])

# Filter for correct barcodes only: ALL binders and peptides
bc=bc[(bc.read1 == 'Bfl1') | (bc.read1=='BclB') | (bc.read1=='Bcl2') | (bc.read1=='BclXL') | (bc.read1=='BclW') | (bc.read1=='Mcl1')]
bc=bc[(bc.read2 == 'F21') | (bc.read2=='F30D') | (bc.read2=='B+') | (bc.read2=='B-CDP01') | (bc.read2=='B40A') | (bc.read2=='2+') | (bc.read2=='4LVT') | (bc.read2=='X-CDP07') | (bc.read2=='MINDI') | (bc.read2=='Bim') | (bc.read2=='Noxa') | (bc.read2=='Puma') | (bc.read2=='Bad') | (bc.read2=='Bik') | (bc.read2=='Hrk') | (bc.read2=='Bmf')]

In [6]:
# replace barcodes with binder names
tags1=['Bcl2','BclB','BclXL','Bfl1','BclW','Mcl1']
tags2=['2+','B+','4LVT','B-CDP01','B40A','F21','F30D','MINDI','X-CDP07','Bad','Bik','Bim','Bmf','Hrk','Noxa','Puma']
tags= tags1 + tags2
df2={}

for tag1 in tags1:
     df2[tag1]=Counter()
for i in range(len(bc)):
    tag1=bc['read1'].iloc[i]
    tag2=bc['read2'].iloc[i]
    df2[tag1][tag2] +=1

In [7]:
#Reorder for binders and peptides
#If binders or peptides were not included in this run, that portion of the matrix should give very low counts, which 
#can be ingnored for analysis 
df3 = pd.DataFrame(df2).fillna(0)
df3=df3[['Bfl1','BclB','Bcl2','BclXL','BclW','Mcl1']]
df3=df3.reindex(['F21','F30D','B+','B-CDP01','B40A','2+','4LVT','X-CDP07','MINDI','Bim','Noxa','Puma','Bad','Bik','Hrk','Bmf'])
df3 = pd.DataFrame(df3).fillna(0)

print df3

            Bfl1     BclB    Bcl2  BclXL    BclW    Mcl1
F21      32801.0   1534.0   133.0   15.0   468.0   461.0
F30D     19275.0    866.0    74.0   73.0   230.0   141.0
B+        1258.0  39055.0   182.0   17.0   393.0   290.0
B-CDP01   5905.0  13644.0   211.0   32.0   593.0  3882.0
B40A      2348.0  25047.0    55.0   95.0   340.0   239.0
2+         233.0     81.0  1009.0   79.0   575.0    32.0
4LVT       199.0     92.0   360.0   16.0  1151.0    45.0
X-CDP07    631.0   7325.0    88.0  115.0  9910.0  1991.0
MINDI      402.0   1375.0    62.0    3.0   163.0  6521.0
Bim         81.0      8.0     1.0   36.0    32.0    81.0
Noxa         0.0      0.0     0.0    0.0     0.0     0.0
Puma        35.0      2.0     0.0    0.0     0.0     0.0
Bad          0.0      0.0     0.0    0.0     0.0     0.0
Bik          0.0      0.0     0.0    0.0     0.0     0.0
Hrk          0.0      0.0     0.0    0.0     0.0     0.0
Bmf          0.0      0.0     1.0    0.0     0.0     0.0


In [8]:
#Save the dataframe!
df3.to_csv(name+'.csv')